# Bellevue University
## Spring 2022
### DSC 650 : Big Data
### Week 6 : Machine Learning Fundamentals

#### Assignment 6.1
04/21/2022
##### By: Yograj Karki

Assignment 6.1 <br>
Using section 5.1 in Deep Learning with Python as a guide (listing 5.3 in particular), create a ConvNet model that classifies images in the MNIST digit dataset. Save the model, predictions, metrics, and validation plots in the dsc650/assignments/assignment06/results directory. If you are using JupyterHub, you can include those plots in your Jupyter notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils

np.random.seed(35)

AttributeError: module 'snappy' has no attribute 'compress'

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

In [ ]:
plt.imshow(X_train[0], cmap='gray')
plt.title(y_train[0])

In [ ]:
X_train = X_train.reshape(60000,784)
X_test = X_test.reshape(10000,784)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

In [ ]:
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]

In [ ]:
#shuffle the training set
for _ in range(5): 
    indexes = np.random.permutation(len(X_train))
    
X_train = X_train[indexes]
Y_train = Y_train[indexes]

#set aside 10,000 for validation
val_images = X_train[:10000,:]
val_labels = Y_train[:10000,:]

# leave rest in training set
train_images = X_train[10000:,:]
train_labels = Y_train[10000:,:]

train_images.shape, val_images.shape

In [ ]:
model = Sequential()

model.add(Dense(512, input_dim=784))
# An "activation" is just a non-linear function applied to the output
# of the layer above. Here, with a "rectified linear unit",
# we clamp all values below 0 to 0.
model.add(Activation('relu'))
# Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(10))
# This special "softmax" activation among other things,
# ensures the output is a valid probaility distribution, that is
# that its values are all non-negative and sum to 1.
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=5, batch_size=128,
             validation_data=(val_images, val_labels))

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
epochs = range(1, len(history.history['loss']) + 1)

In [ ]:
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
plt.savefig('results/6_1_lossplot.png')

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']


In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)

In [ ]:
plt.plot(epochs, train_accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'g', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
plt.savefig('results/6_1_accuracyplot.png')

In [ ]:
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

In [ ]:
predictions = np.argmax(model.predict(X_test), axis=1)

predictions = list(predictions)
actuals = list(y_test)

pred_res = pd.DataFrame({'Actual': actuals, 'Predictions': predictions})
pred_res.to_csv('results/6_1_predictions.csv', index=False)
print (pred_res)

In [ ]:
# save model
model.save('results/6_1_model.h5')

In [ ]:
#Metrics output
with open('results/6_1_metrics.txt', 'w') as f:
    f.write('Training Loss: {}'.format(str(history.history['loss'])))
    f.write('\nTraining Accuracy: {}'.format(str(history.history['accuracy'])))
    f.write('\nTest Loss: {}'.format(score[0]))
    f.write('\nTest Accuracy: {}'.format(score[1]))